EDA for Smile on 60+ project at NSS.
Created by Chris Mulvey on 30 Nov 2020.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
smile = pd.read_csv('../data/SmileOn11.28.2020.csv', skiprows = 2, encoding = "ISO-8859-1")

In [ ]:
smile

Taking the columns that are highlighted in the data dictionary and putting them in a new dataframe.

In [ ]:
# initializing an empty dataframe and putting in one column to test.
smile_small = pd.DataFrame()
smile_small['ID'] = smile['CallerNum']

In [ ]:
smile_small.head()

In [ ]:
smile_small['report_type'] = smile['ReportVersion']

In [ ]:
smile_small['call_date'] = smile['CallDateAndTimeStart']

In [ ]:
smile_small['last_screening'] = smile['SMILE ON 60+ Screening - Last Screening Date']

In [ ]:
smile_small['care_provided'] = smile['SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply)']

In [ ]:
smile_small['enroller_id'] = smile['SMILE ON 60+ Site Information - Enroller ID']

In [ ]:
smile_small['plan_completed'] = smile['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?']

In [ ]:
smile_small['plan_developed'] = smile['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?']

In [ ]:
smile_small['function_restored'] = smile['SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)']

In [ ]:
smile_small.head()

If each row is a separate call to the call center, there are a lot of calls that don't lead 

In [ ]:
smile_small.info()

In [ ]:
smile_small['call_date'] = pd.to_datetime(smile_small['call_date'])

In [ ]:
smile_small.info()

Counts of function restored to see how many rows were marked either yes or no.

In [ ]:
smile_small['function_restored'].value_counts()

Counts of plan completed to see how many were marked yes or no.

In [ ]:
smile_small['plan_completed'].value_counts()

Counts of calls related to clinic using enroller id.

In [ ]:
smile_small['enroller_id'].value_counts()

Counts of calls by individual id number.

In [ ]:
smile_small['ID'].value_counts()

Turning value count of calls by ID into a dataframe.

In [ ]:
call_count = smile_small['ID'].value_counts().rename_axis('ID').reset_index(name='counts')

In [ ]:
call_count.info()

In [ ]:
call_count.head()

Subsetting dataframe or values counts of calls to only keep ids that have one call.

In [ ]:
calls = call_count[call_count['counts'] == 1]

In [ ]:
calls

In [ ]:
joined_calls = smile_small.merge(calls, right_on = 'ID', left_on = 'ID', how = 'inner')

In [ ]:
joined_calls

In [ ]:
joined_calls['care_provided'].value_counts()

In [ ]:
smile_2441776 = smile_small[smile_small['ID'] == 2441776]

In [ ]:
smile_2441776.sort_values(by = 'call_date')

In [ ]:
smile_2441776

Taking columns related to transportation and seeing what that looks like.

In [ ]:
transport = pd.DataFrame()

In [ ]:
transport['ID'] = smile['CallerNum']
transport['barriers'] = smile['SMILE ON 60+ Case Management Information - Transportation Barriers']
transport['provided'] = smile['SMILE ON 60+ Clinic Information - Was initial transportation assistance or referral provided?']
transport['arranged'] = smile['SMILE ON 60+ Verification of Eligibility - Transportation Arranged']

In [ ]:
transport

In [ ]:
transport['barriers'].value_counts()

In [ ]:
transport['provided'].value_counts()

In [ ]:
transport['arranged'].value_counts()

Seeing if there is any difference between clinical or registration

In [ ]:
registration = smile_small[smile_small['report_type'] == 'SMILE ON 60+ Registration']

In [ ]:
registration.head()

In [ ]:
registration.info()

In [ ]:
clinical = smile_small[smile_small['report_type'] == 'SMILE ON 60+ Clinical']

In [ ]:
clinical.head()

In [ ]:
clinical.info()

Trying to see if I can calculate registration to treatment plan completion times.

In [ ]:
# smile_small = smile_small.sort_values(['ID', 'call_date'])
# treatment_time = pd.DataFrame()
# for id in smile_small:
#     smile_small.groupby('ID')
#     if id['report_type'] == 'SMILE ON 60+ Registration':
#         treatment_time.append(id)
#     elif id['plan_completed'] == 'Yes':
#         treatment_time.append(id)
#     smile_small['treatment_time'] = smile_small['call_date'] - smile_small['']

In [ ]:
reg_date = pd.DataFrame()

In [ ]:
# for row_index, row_values in smile_small.iterrows():
#     if row_values['report_type'] == 'SMILE ON 60+ Registration':
#         reg_date.append(row_values['ID'])
#         reg_date.append(row_values['call_date'])

In [ ]:
reg_date = smile_small[smile_small['report_type'] == 'SMILE ON 60+ Registration']

In [ ]:
reg_date.head()

In [ ]:
reg_date.info()

In [ ]:
smile_smaller = pd.DataFrame()

In [ ]:
smile_smaller['ID'] = smile_small['ID']
smile_smaller['report_type'] = smile_small['report_type']
smile_smaller['enroller_id'] = smile_small['enroller_id']
smile_smaller['call_date'] = smile_small['call_date']
smile_smaller['plan_completed'] = smile_small['plan_completed']

In [ ]:
smile_smaller.head()

In [ ]:
# Dropping rows with -1 and -2 as ID.
smile_smaller = smile_smaller[smile_smaller['ID'] > 0]

In [ ]:
smile_smaller.head()

In [ ]:
# Setting a multi index to smile_smaller.
smile_smaller.set_index(['ID', 'report_type'], inplace = True)

In [ ]:
smile_smaller

In [ ]:
# Selecting one ID to see what it looks like.
smile_smaller.loc[(2384106, ['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical']), 'call_date']

In [ ]:
smile_smaller = smile_smaller.sort_index()

In [ ]:
# Running this without the index being sorted gave me a performance warning.
test = smile_smaller.loc[(2384106, 'SMILE ON 60+ Registration'), 'call_date']

In [ ]:
test

In [ ]:
smile_smaller.index.is_lexsorted()

In [ ]:
smile_smaller

In [ ]:
# This code takes the .loc of one index and subtracts the oldest date from the newest date in the index and
# returns a time delta. 
diff1 = smile_smaller.loc[(2384106, ['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical']),
                                          'call_date'].max() - smile_smaller.loc[(2384106,
                                                                                  ['SMILE ON 60+ Registration',
                                                                                   'SMILE ON 60+ Clinical']),
                                                                                 'call_date'].min()

In [ ]:
diff2 = smile_smaller.loc[(2441776, ['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical']),
                                          'call_date'].max() - smile_smaller.loc[(2441776,
                                                                                  ['SMILE ON 60+ Registration',
                                                                                   'SMILE ON 60+ Clinical']),
                                                                                 'call_date'].min()

In [ ]:
diff1

In [ ]:
diff2

An attempt at making a function from the above time calculation.

In [ ]:
# def time_calc(df):
#     for index in df:
#         diff = df.loc[(index, ['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical']), 'call_date'].max() - df.loc[(index, ['SMILE ON 60+ Registration', 'SMILE ON 60+ Clinical']), 'call_date'].min()
#     return diff

In [ ]:
#difference = time_calc(smile_smaller)

After trying the function, I decided on a for loop, using iterrows. The i in the code instead of using an index number as above allows the index to change as the for loop runs.

In [ ]:
diff = []
for i, row in smile_smaller.iterrows():
    diff.append(smile_smaller.loc[(i, ['SMILE ON 60+ Registration', 
                                      'SMILE ON 60+ Clinical']), 
                                 'call_date'].max() - smile_smaller.loc[(i, 
                                                                         ['SMILE ON 60+ Registration', 
                                                                          'SMILE ON 60+ Clinical']), 
                                                                        'call_date'].min())

In [ ]:
diff

In [ ]:
smile_smaller['total_time'] = diff

In [ ]:
smile_smaller.head(20)

In [ ]:
#smile_smaller.to_csv('../data/smile_smaller.csv')

In [ ]:
smile_smaller['total_time'].mean()

In [ ]:
smile_smaller['total_time'].describe()

Looking to see if the duplicate times in total_time are skewing the statistical information.

In [ ]:
smile_smaller = smile_smaller.drop_duplicates(subset=['total_time'])

In [ ]:
smile_smaller.head()

In [ ]:
smile_smaller.mean()

In [ ]:
smile_smaller['total_time'].describe()

In [ ]:
smile_smaller.info()

In [ ]:
smile_smaller['total_time'] = smile_smaller['total_time'].dt.days

In [ ]:
smile_smaller.to_csv('../data/smile_smaller.csv')

In [ ]:
smile_smaller.head()

It appears that having the addditional rows of numbers changed the statistical information. I also noticed that dropping the IDs that were -1 and -2 reduced the number of IDs by almost 2600.

In [ ]:
smile_smaller.hist(column = 'total_time')

In [ ]:
clinic_avg = smile_smaller.groupby('enroller_id').mean()

In [ ]:
clinic_avg

In [ ]:
clinic_avg.to_csv('../data/clinic_avg.csv')

In [ ]:
clinic_avg.plot.bar()

In [ ]:
sns.boxplot(smile_smaller['total_time'],
           orient = 'v')

In [ ]:
sns.boxplot(clinic_avg['total_time'], 
           orient = 'v')

In [ ]:
smile_smaller